In [1]:
# 读取 ../ckpts/bilstm_crf.pkl
import pickle
import sys
sys.path.append("..")  # 将上级目录添加到Python路径
from models.bilstm_crf import BILSTM_Model
with open('../ckpts/bilstm_crf.pkl', 'rb') as fr:
    model = pickle.load(fr)



In [2]:
from utils import load_model, extend_maps, prepocess_data_for_lstmcrf
from data import build_corpus
from evaluating import Metrics
from evaluate import ensemble_evaluate


In [3]:
from os.path import join
from codecs import open

In [4]:
def build_corpus2(make_vocab=True):

    word_lists = []
    tag_lists = []
    with open(join("../ResumeNER", "try.char.bmes"), 'r', encoding='utf-8') as f:
        word_list = []
        tag_list = []
        for line in f:
            if line != '\r\n':
                word, tag = line.strip('\n').split()
                word_list.append(word)
                tag_list.append(tag)
            else:
                word_lists.append(word_list)
                tag_lists.append(tag_list)
                word_list = []
                tag_list = []

    # 如果make_vocab为True，还需要返回word2id和tag2id
    if make_vocab:
        word2id = build_map(word_lists)
        tag2id = build_map(tag_lists)
        return word_lists, tag_lists, word2id, tag2id
    else:
        return word_lists, tag_lists



In [5]:
try_word_lists , try_tag_lists = build_corpus2(make_vocab=False)

In [6]:
try_word_lists

[['耿', '佃', '杰', '先', '生', '，'],
 ['1',
  '9',
  '5',
  '3',
  '年',
  '9',
  '月',
  '出',
  '生',
  '，',
  '研',
  '究',
  '生',
  '学',
  '历',
  '，',
  '高',
  '级',
  '经',
  '济',
  '师',
  '，'],
 ['1',
  '9',
  '7',
  '2',
  '年',
  '参',
  '加',
  '工',
  '作',
  '，',
  '曾',
  '任',
  '海',
  '军',
  '航',
  '空',
  '兵',
  '1',
  '2',
  '师',
  '军',
  '械',
  '师',
  '，',
  '南',
  '京',
  '海',
  '军',
  '学',
  '院',
  '航',
  '空',
  '兵',
  '教',
  '研',
  '室',
  '正',
  '连',
  '职',
  '教',
  '官',
  '、',
  '淄',
  '博',
  '市',
  '化',
  '工',
  '公',
  '司',
  '团',
  '委',
  '副',
  '书',
  '记',
  '、',
  '山',
  '东',
  '张',
  '店',
  '化',
  '工',
  '厂',
  '工',
  '会',
  '主',
  '席',
  '、',
  '常',
  '务',
  '副',
  '厂',
  '长',
  '，',
  '山',
  '东',
  '东',
  '大',
  '化',
  '学',
  '工',
  '业',
  '（',
  '集',
  '团',
  '）',
  '公',
  '司',
  '董',
  '事',
  '长',
  '、',
  '总',
  '经',
  '理',
  '兼',
  '党',
  '委',
  '副',
  '书',
  '记',
  '，',
  '淄',
  '博',
  '市',
  '电',
  '子',
  '工',
  '业',
  '公',
  '司',
  '党',
  '委',
  '书',
  '记',
  '兼',
  '经',


In [13]:
try_tag_lists

[['B-NAME', 'M-NAME', 'E-NAME', 'O', 'O', 'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-EDU',
  'M-EDU',
  'M-EDU',
  'M-EDU',
  'E-EDU',
  'O',
  'B-TITLE',
  'M-TITLE',
  'M-TITLE',
  'M-TITLE',
  'E-TITLE',
  'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-ORG',
  'M-ORG',
  'M-ORG',
  'M-ORG',
  'M-ORG',
  'M-ORG',
  'M-ORG',
  'E-ORG',
  'B-TITLE',
  'M-TITLE',
  'E-TITLE',
  'O',
  'B-ORG',
  'M-ORG',
  'M-ORG',
  'M-ORG',
  'M-ORG',
  'M-ORG',
  'M-ORG',
  'M-ORG',
  'M-ORG',
  'M-ORG',
  'M-ORG',
  'E-ORG',
  'B-TITLE',
  'M-TITLE',
  'M-TITLE',
  'M-TITLE',
  'E-TITLE',
  'O',
  'B-ORG',
  'M-ORG',
  'M-ORG',
  'M-ORG',
  'M-ORG',
  'M-ORG',
  'E-ORG',
  'B-TITLE',
  'M-TITLE',
  'M-TITLE',
  'M-TITLE',
  'E-TITLE',
  'O',
  'B-ORG',
  'M-ORG',
  'M-ORG',
  'M-ORG',
  'M-ORG',
  'M-ORG',
  'E-ORG',
  'B-TITLE',
  'M-TITLE',
  'M-TITLE',
  'E-TITLE',
  'O',
  'B-TITLE',
  'M-TITLE',
  'M-TITLE',
 

In [8]:
def build_map(lists):
    maps = {}
    for list_ in lists:
        for e in list_:
            if e not in maps:
                maps[e] = len(maps)

    return maps

In [9]:
def build_corpus3(make_vocab=True):

    word_lists = []
    tag_lists = []
    with open(join("../ResumeNER", "train.char.bmes"), 'r', encoding='utf-8') as f:
        word_list = []
        tag_list = []
        for line in f:
            if line != '\r\n':
                word, tag = line.strip('\n').split()
                word_list.append(word)
                tag_list.append(tag)
            else:
                word_lists.append(word_list)
                tag_lists.append(tag_list)
                word_list = []
                tag_list = []

    # 如果make_vocab为True，还需要返回word2id和tag2id
    if make_vocab:
        word2id = build_map(word_lists)
        tag2id = build_map(tag_lists)
        return word_lists, tag_lists, word2id, tag2id
    else:
        return word_lists, tag_lists


In [10]:
train_word_lists, train_tag_lists, word2id, tag2id = \
        build_corpus3("train")

In [14]:
crf_word2id, crf_tag2id = extend_maps(word2id, tag2id, for_crf=True)

In [15]:
crf_word2id


{'高': 0,
 '勇': 1,
 '：': 2,
 '男': 3,
 '，': 4,
 '中': 5,
 '国': 6,
 '籍': 7,
 '无': 8,
 '境': 9,
 '外': 10,
 '居': 11,
 '留': 12,
 '权': 13,
 '1': 14,
 '9': 15,
 '6': 16,
 '年': 17,
 '出': 18,
 '生': 19,
 '汉': 20,
 '族': 21,
 '共': 22,
 '党': 23,
 '员': 24,
 '本': 25,
 '科': 26,
 '学': 27,
 '历': 28,
 '工': 29,
 '程': 30,
 '师': 31,
 '、': 32,
 '美': 33,
 '项': 34,
 '目': 35,
 '管': 36,
 '理': 37,
 '协': 38,
 '会': 39,
 '注': 40,
 '册': 41,
 '（': 42,
 'P': 43,
 'M': 44,
 'I': 45,
 'e': 46,
 'm': 47,
 'b': 48,
 'r': 49,
 '）': 50,
 '专': 51,
 '家': 52,
 '经': 53,
 '。': 54,
 '2': 55,
 '0': 56,
 '7': 57,
 '月': 58,
 '至': 59,
 '今': 60,
 '任': 61,
 '人': 62,
 '和': 63,
 '投': 64,
 '资': 65,
 '董': 66,
 '事': 67,
 '；': 68,
 '3': 69,
 '公': 70,
 '司': 71,
 '秘': 72,
 '书': 73,
 '综': 74,
 '合': 75,
 '部': 76,
 '长': 77,
 '山': 78,
 '东': 79,
 '三': 80,
 '维': 81,
 '石': 82,
 '化': 83,
 '股': 84,
 '份': 85,
 '有': 86,
 '限': 87,
 '副': 88,
 '总': 89,
 '张': 90,
 '雁': 91,
 '冰': 92,
 '5': 93,
 '永': 94,
 '久': 95,
 '兰': 96,
 '州': 97,
 '商': 98,
 '院': 99,
 '计': 100,

In [16]:
crf_tag2id


{'B-NAME': 0,
 'E-NAME': 1,
 'O': 2,
 'B-CONT': 3,
 'M-CONT': 4,
 'E-CONT': 5,
 'B-RACE': 6,
 'E-RACE': 7,
 'B-TITLE': 8,
 'M-TITLE': 9,
 'E-TITLE': 10,
 'B-EDU': 11,
 'M-EDU': 12,
 'E-EDU': 13,
 'B-ORG': 14,
 'M-ORG': 15,
 'E-ORG': 16,
 'M-NAME': 17,
 'B-PRO': 18,
 'M-PRO': 19,
 'E-PRO': 20,
 'S-RACE': 21,
 'S-NAME': 22,
 'B-LOC': 23,
 'M-LOC': 24,
 'E-LOC': 25,
 'M-RACE': 26,
 'S-ORG': 27,
 '<unk>': 28,
 '<pad>': 29,
 '<start>': 30,
 '<end>': 31}

In [22]:
 bilstm_model = load_model('../ckpts/bilstm_crf.pkl')

In [23]:
bilstm_model.model.bilstm.bilstm.flatten_parameters()  # remove warning


In [24]:
try_word_lists, try_tag_lists = prepocess_data_for_lstmcrf(
        try_word_lists, try_tag_lists, test=True
    )

In [25]:
lstmcrf_pred, target_tag_list = bilstm_model.test(try_word_lists, try_tag_lists,
                                                  crf_word2id, crf_tag2id)


In [26]:
target_tag_list


[['B-NAME', 'M-NAME', 'E-NAME', 'O', 'O', 'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-EDU',
  'M-EDU',
  'M-EDU',
  'M-EDU',
  'E-EDU',
  'O',
  'B-TITLE',
  'M-TITLE',
  'M-TITLE',
  'M-TITLE',
  'E-TITLE',
  'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-ORG',
  'M-ORG',
  'M-ORG',
  'M-ORG',
  'M-ORG',
  'M-ORG',
  'M-ORG',
  'E-ORG',
  'B-TITLE',
  'M-TITLE',
  'E-TITLE',
  'O',
  'B-ORG',
  'M-ORG',
  'M-ORG',
  'M-ORG',
  'M-ORG',
  'M-ORG',
  'M-ORG',
  'M-ORG',
  'M-ORG',
  'M-ORG',
  'M-ORG',
  'E-ORG',
  'B-TITLE',
  'M-TITLE',
  'M-TITLE',
  'M-TITLE',
  'E-TITLE',
  'O',
  'B-ORG',
  'M-ORG',
  'M-ORG',
  'M-ORG',
  'M-ORG',
  'M-ORG',
  'E-ORG',
  'B-TITLE',
  'M-TITLE',
  'M-TITLE',
  'M-TITLE',
  'E-TITLE',
  'O',
  'B-ORG',
  'M-ORG',
  'M-ORG',
  'M-ORG',
  'M-ORG',
  'M-ORG',
  'E-ORG',
  'B-TITLE',
  'M-TITLE',
  'M-TITLE',
  'E-TITLE',
  'O',
  'B-TITLE',
  'M-TITLE',
  'M-TITLE',
 

In [27]:
lstmcrf_pred


[['B-NAME', 'M-NAME', 'E-NAME', 'O', 'O', 'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-EDU',
  'M-EDU',
  'M-EDU',
  'M-EDU',
  'E-EDU',
  'O',
  'B-TITLE',
  'M-TITLE',
  'M-TITLE',
  'M-TITLE',
  'E-TITLE',
  'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-ORG',
  'M-ORG',
  'M-ORG',
  'M-ORG',
  'M-ORG',
  'M-ORG',
  'M-ORG',
  'M-ORG',
  'B-TITLE',
  'M-TITLE',
  'E-TITLE',
  'O',
  'B-ORG',
  'M-ORG',
  'M-ORG',
  'M-ORG',
  'M-ORG',
  'M-ORG',
  'M-ORG',
  'M-ORG',
  'M-ORG',
  'M-ORG',
  'M-ORG',
  'E-ORG',
  'B-TITLE',
  'M-TITLE',
  'M-TITLE',
  'M-TITLE',
  'E-TITLE',
  'O',
  'B-ORG',
  'M-ORG',
  'M-ORG',
  'M-ORG',
  'M-ORG',
  'M-ORG',
  'E-ORG',
  'B-TITLE',
  'M-TITLE',
  'M-TITLE',
  'M-TITLE',
  'E-TITLE',
  'O',
  'B-ORG',
  'M-ORG',
  'M-ORG',
  'M-ORG',
  'M-ORG',
  'M-ORG',
  'E-ORG',
  'B-TITLE',
  'M-TITLE',
  'M-TITLE',
  'E-TITLE',
  'O',
  'B-TITLE',
  'M-TITLE',
  'M-TITLE',
 

In [32]:
REMOVE_O = False

In [33]:
metrics = Metrics(target_tag_list, lstmcrf_pred, remove_O=REMOVE_O)
metrics.report_scores()
metrics.report_confusion_matrix()

           precision    recall  f1-score   support
   E-NAME     1.0000    1.0000    1.0000         1
    E-EDU     1.0000    1.0000    1.0000         1
   M-NAME     1.0000    1.0000    1.0000         1
  B-TITLE     1.0000    1.0000    1.0000        13
    B-ORG     1.0000    1.0000    1.0000         7
  M-TITLE     1.0000    1.0000    1.0000        25
    E-ORG     1.0000    0.8571    0.9231         7
    M-ORG     0.9787    1.0000    0.9892        46
    B-EDU     1.0000    1.0000    1.0000         1
    M-EDU     1.0000    1.0000    1.0000         3
   B-NAME     1.0000    1.0000    1.0000         1
  E-TITLE     1.0000    1.0000    1.0000        13
        O     1.0000    1.0000    1.0000        41
avg/total     0.9939    0.9938    0.9935       160

Confusion Matrix:
         E-NAME   E-EDU  M-NAME B-TITLE   B-ORG M-TITLE   E-ORG   M-ORG   B-EDU   M-EDU  B-NAME E-TITLE       O 
 E-NAME       1       0       0       0       0       0       0       0       0       0       0       0